# Initialisation

In [ ]:
import write_files as wf
import lpse_data as ld
import random_srs as rs
import matplotlib.pyplot as plt
import numpy as np
import copy
import pickle
import ihp
from functools import partial
from time import time as stopwatch
import calc_inputs as ci
import scipy.constants as scc
import ihp

# Ipython magic features
%load_ext autoreload
%autoreload 2
  
# LPSE class
lpse = ld.lpse_case()
lpse.dfp = './data/lpse.' # Data file prefix
lpse.verbose = False # Show prints
lpse.np = 1 # Number of processors
lpse.bin = '/home/space/phrfqm/lpse-3.2.11/bin/lpse_cpu' # Binary

# Base case setup

In [ ]:
jc = wf.job_control()
jc.version = '3.2.11' 
jc.seed = 1 # 0 for random, otherwise fixed seed
jc.resources.heartbeatInterval = 0.1 # minutes
lpse.add_class(jc)

In [ ]:
gr = wf.gridding()
gr.grid.sizes = 13 # microns
gr.grid.nodes = 4321
gr.grid.antiAliasing.range = 0.333
gr.grid.antiAliasing.isAutomatic = 'false'
lpse.add_class(gr)

In [ ]:
cm = wf.components()
cm.laser.enable = 'true'
cm.raman.enable = 'true'
cm.lw.enable = 'true'
cm.iaw.enable = 'false'
lpse.add_class(cm)

In [ ]:
tc = wf.temporal_control()
tc.simulation.samplePeriod = 0.01 # ps
tc.simulation.time.end = 2 # ps
lpse.add_class(tc)

In [ ]:
io = wf.io_control()
io.grid.downSampleFactors = 4 # Spatial
io.laser.save.S0.x = lpse.dfp + 'S0_x'
io.raman.save.S0.x = lpse.dfp + 'S1_x'
io.laser.save.E0.z = lpse.dfp + 'E0_z'
io.raman.save.E0.z = lpse.dfp + 'E1_z'
io.lw.save.pots = lpse.dfp + 'pots'
lpse.add_class(io)

In [ ]:
pp = wf.physical_parameters()
pp.physical.Z = 1
pp.physical.Te = 4.5 # keV
pp.physical.Ti = 4.5 # keV
pp.physical.MiOverMe =  1836.15
pp.lw.envelopeDensity = 0.15
lpse.plasmaFrequencyDensity = 0.15
pp.densityProfile.shape = 'exp'
pp.densityProfile.geometry = 'cartesian'
pp.densityProfile.NminOverNc = 0.1357
pp.densityProfile.NmaxOverNc = 0.1658
pp.densityProfile.NminLocation = '-50 0 0'
pp.densityProfile.NmaxLocation = '50 0 0'
lpse.add_class(pp)

In [ ]:
lc = wf.light_control()
lc.laser.wavelength = 0.351 # microns
lc.laser.pumpDepletion.SRS.enable = 'true'
lc.laser.ionAcousticPerturbations.enable = 'true'
lc.laser.evolution.Labc = 3 # microns
lc.laser.evolution.Loff = 1 # microns
lc.laser.evolution.riseTime = 30
lc.laser.evolution.absorption = 0
lc.raman.sourceTerm.lw.enable = 'true'
lc.raman.ionAcousticPerturbations.enable = 'true'
lc.raman.evolution.Labc = 3
lc.raman.evolution.Loff = 1 
lc.raman.evolution.riseTime = 30
lc.raman.evolution.absorption = 0
# fd solvers
lc.laser.solver = 'fd'
lc.laser.evolution.solverOrder = 2
lc.laser.evolution.dtFraction = 0.95
lc.raman.solver = 'fd'
lc.raman.evolution.solverOrder = 2
lc.raman.evolution.dtFraction = 0.95
# spectral solvers
# lc.laser.solver = 'spectral'
# lc.laser.spectral.dt = 2e-6 # ps
# lc.raman.solver = 'spectral'
# lc.raman.spectral.dt = 2e-6
lpse.add_class(lc)

In [ ]:
ls = wf.light_source()
ls.laser.nBeams = 1
ls.laser.intensity = ['1e+15'] # W/cm^2
ls.laser.phase = [0] # degrees
ls.laser.polarization = [90] # degrees
ls.laser.direction = ['1 0 0']
ls.laser.frequencyShift = [0]
ls.laser.group = [0]
ls.laser.evolution.source = ['min.x']
ls.laser.evolution.offset = ['0 0 0'] # microns
ls.laser.evolution.width = [10] # Half-width at 1/e of sgauss [um]
ls.laser.evolution.sgOrder = [4]
ls.raman.nBeams = 1
ls.raman.intensity = ['6.155e+10' for i in range(ls.raman.nBeams)] # W/cm^2
ls.raman.phase = [0 for i in range(ls.raman.nBeams)] # degrees
ls.raman.polarization = [90 for i in range(ls.raman.nBeams)] # degrees
ls.raman.direction = ['-1 0 0' for i in range(ls.raman.nBeams)]
ls.raman.frequencyShift = [0]
ls.raman.group = [0]
ls.raman.evolution.source = ['max.x' for i in range(ls.raman.nBeams)]
ls.raman.evolution.offset = ['0 0 0' for i in range(ls.raman.nBeams)] # microns
ls.raman.evolution.width = [0 for i in range(ls.raman.nBeams)] # Half-width at 1/e of sgauss [um]
ls.raman.evolution.sgOrder = [4 for i in range(ls.raman.nBeams)]
lpse.add_class(ls)

In [ ]:
lwc = wf.lw_control()
lwc.lw.SRS.enable = 'true'
lwc.lw.ionAcousticPerturbations.enable = 'true'
lwc.lw.spectral.dt = 2e-6 # ps
lwc.lw.Labc = 3 # microns
lwc.lw.noise.enable = 'false'
lwc.lw.noise.isCalculated = 'false'
lwc.lw.noise.amplitude = 0.001
lwc.lw.collisionalDampingRate = 0.0
lwc.lw.__dict__['collisionalDampingRate.isCalculated'] = 'false'
lwc.lw.landauDamping.enable = 'true'
lwc.lw.kFilter.enable = 'false'
lwc.lw.kFilter.scale = 1.2
lpse.add_class(lwc)

In [ ]:
iawc = wf.iaw_control()
iawc.iaw.solver = 'spectral'
iawc.iaw.sourceTerm.lw.enable = 'true'
iawc.iaw.sourceTerm.laser.enable = 'true'
iawc.iaw.sourceTerm.raman.enable = 'true'
iawc.iaw.spectral.dt = 1e-3
iawc.iaw.Labc = 3
iawc.iaw.dampingRate = 0.0
lpse.add_class(iawc)

In [ ]:
ins = wf.instrumentation()
ins.metrics.enable = 'true'
ins.metrics.file = lpse.dfp + 'metrics'
ins.metrics.samplePeriod = 0.01 # ps
lpse.add_class(ins)

In [ ]:
pert = wf.initial_perturbation()
pert.initialPerturbation.enable = 'true'
pert.initialPerturbation.field = 'E1_z'
pert.initialPerturbation.type = 'planeWave'
pert.initialPerturbation.wavelength = 1 # set after calcs
pert.initialPerturbation.direction = '[-1 0 0]'
pert.initialPerturbation.envelopeSize = '[0 0 0]' # infinite
pert.initialPerturbation.envelopeOffset = '[0 0 0]'
pert.initialPerturbation.amplitude = 1 # set after calcs
lpse.add_class(pert)

# List of density profile classes

In [ ]:
# format ppxy where x = L_n, y = n_mid, and 1,2,3 = low, mid, high
pp22 = copy.deepcopy(pp)
pp12 = copy.deepcopy(pp)
pp12.densityProfile.NminOverNc = 0.1270
pp12.densityProfile.NmaxOverNc = 0.1772
pp21 = copy.deepcopy(pp)
pp21.lw.envelopeDensity = 0.12
pp21.densityProfile.NminOverNc = 0.1086
pp21.densityProfile.NmaxOverNc = 0.1326
pp23 = copy.deepcopy(pp)
pp23.lw.envelopeDensity = 0.20
pp23.densityProfile.NminOverNc = 0.181
pp23.densityProfile.NmaxOverNc = 0.221
pp32 = copy.deepcopy(pp)
pp32.densityProfile.NminOverNc = 0.1427
pp32.densityProfile.NmaxOverNc = 0.1577
dens = [pp12,pp21,pp22,pp23,pp32]
I14s = [7.956e10,5.822e10,7.341e10,9.437e10,6.451e10]
dlabs = ['Ln=300um; nmid=0.15','Ln=500um; nmid=0.12',\
         'Ln=500um; nmid=0.15','Ln=500um; nmid=0.20',\
         'Ln=1000um; nmid=0.15']

In [ ]:
# Single run point
splits = 1; cpw = 8; tavg = tc.simulation.time.end
for i in range(2,3):
  lpse.add_class(dens[i])
#   rs.Isrs14(I14s[i],lpse,tavg,splits,cpw,True,True:)
  isrstest = rs.Isrs(lpse,tavg,splits,cpw,True)

In [ ]:
lpse.fields()

In [ ]:
%matplotlib notebook
lpse.plot_field('S1_x')

# Run cases and get $<I_{srs}>$ curves

In [ ]:
# Isrs curves
fresh_results = False
Irange = np.logspace(14,16,20)
cpus = 2; splits = 3; cpw = 8
tavg = tc.simulation.time.end; parallel = True
if fresh_results:
  t0 = stopwatch()
  isrs = rs.Isrs_dens(lpse,dens[:2],dlabs[:2],tavg,I14s[:2],\
                      Irange,parallel,cpus,splits,cpw)
  t1 = stopwatch()
  print(f'Time taken: {t1-t0:0.3f}s')
  with open('isrsscrap.pickle', 'wb') as f:
    pickle.dump(isrs, f)
else:
  with open('raman_fit_3splits.pickle', 'rb') as f:
    isrs = pickle.load(f)

In [ ]:
# Plot curves
%matplotlib inline
for i in range(len(dlabs)):
  plt.loglog(Irange,isrs[dlabs[i]],label=dlabs[i])
plt.loglog(Irange,Irange,'--',label='Saturation')
plt.ylim(None,1e16)
plt.xlim(1e14,1e16)
plt.legend()
plt.show()

In [ ]:
# Extract experimental datasets
lasI = {}; srsI = {}
srcs = ['epoch','fluid']
vals = ['low','mid','high']
typs = ['scale','dens']
fnames = [f'SJdata/{i}_srs_{j}_{k}.csv' \
          for i in srcs for j in vals for k in typs]
for i,j in enumerate(fnames):
  lasI[i] = []; srsI[i] = []
  with open(j,'r') as fp:
    for line in fp:
      lin = line.strip().split(',')
      lasI[i].append(float(lin[0]))
      srsI[i].append(float(lin[1]))

In [ ]:
# Plot paper figures
%matplotlib inline
fig1 = [0,2,4]; fig2 = [1,2,3]
fig1_epoch = [0,2,4]; fig2_epoch = [1,3,5]
fig1_fluid = [6,8,10]; fig2_fluid = [7,9,11]
labels = [f'{i}_{j}' for i in srcs for j in vals for k in range(2)]
for i in fig1:
  plt.loglog(Irange,isrs[dlabs[i]],label=dlabs[i])
for i in fig1_epoch:
#   print(f'{srsI[i][0]:0.3e}')
  plt.loglog(lasI[i],srsI[i],'x--',label=labels[i])
# for i in fig1_fluid:
#   plt.loglog(lasI[i],srsI[i],'--',label=labels[i])
# plt.loglog(Irange,Irange,'--',label='Saturation')
plt.ylim(4e10,1e16)
plt.xlim(1e14,1e16)
# plt.hlines(2e15,1e14,1e16)
# plt.vlines(1e16,4e10,1e16)
plt.legend(bbox_to_anchor=(1.05, 1))
# plt.grid()
plt.show()
for i in fig2:
  plt.loglog(Irange,isrs[dlabs[i]],label=dlabs[i])
for i in fig2_epoch:
#   print(f'{srsI[i][0]:0.3e}')
  plt.loglog(lasI[i],srsI[i],'x--',label=labels[i])
# for i in fig2_fluid:
#   plt.loglog(lasI[i],srsI[i],'--',label=labels[i])
# plt.loglog(Irange,Irange,'--',label='Saturation')
plt.ylim(None,1e16)
plt.xlim(1e14,1e16)
plt.legend(bbox_to_anchor=(1.05, 1))
# plt.grid()
plt.show()

In [ ]:
# Exp density profile
%matplotlib inline
xmax = 50; xmin = -50
x = np.linspace(xmin,xmax,101)
r = abs(x-xmin)
r = x
ncen = 0.20
Ln = 500
n = ncen*np.exp(r/Ln)
print(n[0],n[-51],n[-1])
# test = np.linspace(0.14,0.17,100)
plt.plot(x,n,label='exp')
# plt.plot(x,test,label='lin')
plt.legend()
plt.show()

In [ ]:
# Analytical solution from literature
from scipy.special import erf
tst = 1j*3
print(erf(tst))
